In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import json

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print("y lable min : {}".format(min(y_train)))
print("y lable max : {}".format(max(y_train)))

In [ ]:
rows = 3
cols = 3
axes=[]
fig=plt.figure()

for a in range(rows*cols):
    axes.append(fig.add_subplot(rows, cols, a+1))
    subplot_title=(y_train[a])
    axes[-1].set_title(subplot_title)
    plt.imshow(x_train[a], cmap="gray")
fig.tight_layout()
plt.show()

# CNN

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(input_shape=(28, 28, 1), filters=64, kernel_size=(3, 3), strides=(1,1), padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(strides=(2,2)))
model.add(tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(strides=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation="relu"))
model.add(tf.keras.layers.Dense(units=128, activation="relu"))
model.add(tf.keras.layers.Dense(units=128, activation="relu"))
model.add(tf.keras.layers.Dense(units=10, activation="softmax"))

model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"]
)

#계층, 차원, 파라미터 수 요약 확인
model.summary()

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
print(x_train.shape)
print(x_test.shape)

In [ ]:
# GPU 메모리 문제 발생 예방
# config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
# config.gpu_options.per_process_gpu_memory_fraction = 0.8
# tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

epoch_counts = 30
history = model.fit(
    x_train,
    y_train,
    epochs=epoch_counts,
    # batch_size = batch_size,
    validation_split=0.25
    # callbacks=[early_stopping]
)

#train loss, validation loss 그래프 출력
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], "b-", label="loss")
plt.plot(history.history["val_loss"], "r--", label="val_loss")
plt.xlabel("Epoch")
plt.legend()

#train accutacy, validation accuracy 그래프 출력
plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"], "g-", label="accuracy")
plt.plot(history.history["val_accuracy"], "k--", label="val_accuracy")
plt.xlabel("Epoch")
plt.ylim(0.7, 1)
plt.legend()
plt.show()

In [ ]:
print("최종 결과 : ", model.evaluate(x_test, y_test))

class_name_dic_by_no = {
    "0":"티셔츠/탑",
    "1":"바지",
    "2":"스웨터",
    "3":"드레스",
    "4":"코트",
    "5":"샌들",
    "6":"셔츠",
    "7":"스니커즈",
    "8":"가방",
    "9":"발목부츠"
}

#모델 저장
try:
  model_dir = "drive/MyDrive/Colab Notebooks/models/mnist_model"
  json_string = model.to_json()
  open(model_dir + "/pgu_mnist_fashion_model_cnn_json.json", "w").write(json_string)
  model.save_weights(model_dir + "/pgu_mnist_fashion_model_cnn_weights.h5", overwrite=True)
  with open(model_dir + "/pgu_mnist_fashion_model_cnn_names.json", "w") as outfile:
      json.dump(class_name_dic_by_no, outfile)
except:
  print("=================================")
  print("=================================")
  print("모델 저장 중 오류가 발생하였습니다!")
  print("=================================")
  print("=================================")
finally:
  pass